# Import package for data processing

In [44]:
import json
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from functools import reduce
from collections import Counter
from nltk.corpus import stopwords
from nltk.corpus import brown
from string import punctuation, digits
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.stem.porter import *
import numpy as np
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

with open("camera_reviews_full.json", 'r') as f:
    js = json.load(f)
    f.close()
gensim.__version__

'3.1.0'

## To run the LDA model for this script, we need to prepare Gensim 3.1.0. If using the version other than 3.1.0, it'll produce NaN and same words in each topic.
    downgrading Gensim in Mac terminal:
       $ pip3 install -Iv gensim==3.1.0

# Exclude punctuations, stopwords, alphabets and digits. And lemmatizing the words.

In [45]:
stopwords_set = set(stopwords.words('english'))
brown_set = set(brown.words())
characterstoclean = r'?!,:,/\"-+=@#$%^&*()><{}[]|' + r"'"
punc = punctuation

In [46]:
def data_processing(post_list):
    words = []
    for word in post_list.split(' '):
        word = word.lower()
        if word in brown_set and len(word) > 2 and word not in stopwords_set and word not in characterstoclean and word not in punc and not word.isdigit() and word.isalpha():
            words.append(word)
    return words

In [47]:
js_lda = js.copy()
for i in js_lda.keys():
    for j in js_lda[i].keys(): 
        if j == "review":
            js_lda[i][j] = data_processing(js_lda[i][j])

# Prepare packages for LDA model

# Create two lists which store verified and unverified customers' reviews separately for running LDA model.

In [5]:
lda_list_verified=[]
lda_list_unverified=[]
for i in js_lda.keys():
    if js_lda[i]["author_attributes"] == []:
        lda_list_unverified.append(js_lda[i]["review"])
    elif "Verified" in js_lda[i]["author_attributes"][0]:
        lda_list_verified.append(js_lda[i]["review"])
    else:
        lda_list_unverified.append(js_lda[i]["review"])

In [6]:
len(lda_list_unverified), len(lda_list_verified)

(31, 3459)

In [7]:
dict_unverified = corpora.Dictionary(lda_list_unverified)
print(dict_unverified)
doc_term_matrix_unverified = [dict_unverified.doc2bow(doc) for doc in lda_list_unverified]
# print(doc_term_matrix_unverified)

Dictionary(339 unique tokens: ['great', 'starter', 'camera', 'comes', 'everything']...)


In [11]:
lda_model_unverified2 = gensim.models.ldamodel.LdaModel(doc_term_matrix_unverified, num_topics=2, id2word = dict_unverified, passes=50)

In [9]:
lda_model_unverified3 = gensim.models.ldamodel.LdaModel(doc_term_matrix_unverified, num_topics=3, id2word = dict_unverified, passes=50)

In [8]:
lda_model_unverified4 = gensim.models.ldamodel.LdaModel(doc_term_matrix_unverified, num_topics=4, id2word = dict_unverified, passes=50)

In [10]:
lda_model_unverified5 = gensim.models.ldamodel.LdaModel(doc_term_matrix_unverified, num_topics=5, id2word = dict_unverified, passes=50)

In [12]:
for idx_unverified, topic_unverified in lda_model_unverified2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_unverified, topic_unverified))

Topic: 0 
Words: 0.035*"camera" + 0.021*"quality" + 0.017*"take" + 0.017*"use" + 0.015*"pictures" + 0.012*"would" + 0.011*"great" + 0.010*"takes" + 0.010*"get" + 0.010*"good"
Topic: 1 
Words: 0.032*"camera" + 0.020*"great" + 0.019*"lens" + 0.013*"easy" + 0.012*"better" + 0.010*"first" + 0.010*"canon" + 0.010*"love" + 0.010*"kit" + 0.010*"need"


In [13]:
for idx_unverified, topic_unverified in lda_model_unverified3.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_unverified, topic_unverified))

Topic: 0 
Words: 0.040*"camera" + 0.020*"great" + 0.016*"pictures" + 0.015*"money" + 0.013*"take" + 0.013*"gift" + 0.013*"takes" + 0.013*"spend" + 0.013*"buy" + 0.013*"easy"
Topic: 1 
Words: 0.035*"camera" + 0.018*"quality" + 0.015*"easy" + 0.015*"lenses" + 0.015*"lens" + 0.015*"take" + 0.012*"amazing" + 0.012*"canon" + 0.012*"kit" + 0.012*"like"
Topic: 2 
Words: 0.028*"camera" + 0.019*"use" + 0.016*"quality" + 0.016*"pictures" + 0.016*"good" + 0.016*"great" + 0.014*"love" + 0.011*"first" + 0.011*"get" + 0.011*"settings"


In [14]:
for idx_unverified, topic_unverified in lda_model_unverified4.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_unverified, topic_unverified))

Topic: 0 
Words: 0.026*"quality" + 0.018*"would" + 0.018*"use" + 0.018*"camera" + 0.013*"first" + 0.013*"pictures" + 0.013*"great" + 0.013*"taking" + 0.013*"using" + 0.013*"easy"
Topic: 1 
Words: 0.020*"use" + 0.020*"gift" + 0.014*"takes" + 0.014*"great" + 0.014*"camera" + 0.014*"pictures" + 0.014*"upgrade" + 0.014*"auto" + 0.014*"take" + 0.014*"card"
Topic: 2 
Words: 0.028*"camera" + 0.020*"amazing" + 0.020*"take" + 0.016*"lens" + 0.016*"pictures" + 0.012*"use" + 0.012*"get" + 0.012*"little" + 0.012*"easy" + 0.009*"well"
Topic: 3 
Words: 0.060*"camera" + 0.021*"great" + 0.018*"lenses" + 0.018*"quality" + 0.015*"good" + 0.013*"love" + 0.013*"price" + 0.013*"better" + 0.013*"came" + 0.010*"easy"


In [15]:
for idx_unverified, topic_unverified in lda_model_unverified5.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_unverified, topic_unverified))

Topic: 0 
Words: 0.033*"quality" + 0.017*"pictures" + 0.017*"taking" + 0.017*"would" + 0.016*"use" + 0.012*"video" + 0.012*"using" + 0.012*"take" + 0.012*"point" + 0.012*"shoot"
Topic: 1 
Words: 0.045*"camera" + 0.018*"amazing" + 0.015*"lens" + 0.015*"take" + 0.015*"great" + 0.012*"love" + 0.012*"bundle" + 0.012*"came" + 0.012*"use" + 0.012*"first"
Topic: 2 
Words: 0.042*"camera" + 0.018*"good" + 0.018*"easy" + 0.018*"better" + 0.018*"canon" + 0.016*"quality" + 0.015*"use" + 0.015*"great" + 0.015*"lot" + 0.015*"really"
Topic: 3 
Words: 0.035*"camera" + 0.018*"good" + 0.018*"buy" + 0.018*"pictures" + 0.018*"need" + 0.018*"everything" + 0.018*"amazing" + 0.018*"great" + 0.010*"got" + 0.010*"wife"
Topic: 4 
Words: 0.028*"great" + 0.015*"takes" + 0.015*"friendly" + 0.015*"user" + 0.015*"excellent" + 0.015*"nature" + 0.015*"gift" + 0.015*"well" + 0.015*"money" + 0.015*"camera"


In [16]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_unverified2 = pyLDAvis.gensim.prepare(lda_model_unverified2, doc_term_matrix_unverified ,dict_unverified)
vis_unverified2

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.060623  0.0       1        1  57.038704
1     -0.060623  0.0       2        1  42.961296, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
61    Default   5.000000        lens   5.000000  30.0000  30.0000
52    Default   3.000000      better   3.000000  29.0000  29.0000
17    Default   3.000000         kit   3.000000  28.0000  28.0000
5     Default   3.000000        need   3.000000  27.0000  27.0000
104   Default   3.000000        well   3.000000  26.0000  26.0000
140   Default   4.000000       would   4.000000  25.0000  25.0000
129   Default   4.000000       takes   4.000000  24.0000  24.0000
0     Default   9.000000       great   9.000000  23.0000  23.0000
14    Default   2.000000        many   2.000000  22.0000  22.0000
205   Default   3.000000      really   3.000000  21.0000  21.0000
203   Default   3.000000         say   3.000000  20.0000  20.0000
8     Default   1.000000        even   1.000000  19.0000  19.0000
41    Default   1.000000        went   1.000000  18.0000  18.0000
1     Default   1.000000     starter   1.000000  17.0000  17.0000
26    Default   1.000000        make   1.000000  16.0000  16.0000
76    Default   1.000000         may   1.000000  15.0000  15.0000
180   Default   1.000000       clear   1.000000  14.0000  14.0000
196   Default   1.000000      bought   1.000000  13.0000  13.0000
117   Default   1.000000       extra   1.000000  12.0000  12.0000
63    Default   1.000000       light   1.000000  11.0000  11.0000
3     Default   1.000000       comes   1.000000  10.0000  10.0000
289   Default   1.000000       loved   1.000000   9.0000   9.0000
316   Default   1.000000         new   1.000000   8.0000   8.0000
119   Default   4.000000       first   4.000000   7.0000   7.0000
40    Default   3.000000        like   3.000000   6.0000   6.0000
118   Default   4.000000        love   4.000000   5.0000   5.0000
7     Default   8.000000        take   8.000000   4.0000   4.0000
100   Default   8.000000         use   8.000000   3.0000   3.0000
53    Default  10.000000     quality  10.000000   2.0000   2.0000
147   Default   2.000000      taking   2.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
67     Topic2   0.951025     connect   1.295737   0.5356  -5.7141
74     Topic2   0.951025    downfall   1.295737   0.5356  -5.7141
11     Topic2   0.951025        room   1.295737   0.5356  -5.7141
24     Topic2   0.951025       savvy   1.295737   0.5356  -5.7141
16     Topic2   0.951025     stories   1.295737   0.5356  -5.7141
88     Topic2   0.951025  reasonable   1.295737   0.5356  -5.7141
40     Topic2   2.234631        like   3.251740   0.4698  -4.8598
119    Topic2   2.907279       first   4.571622   0.3922  -4.5967
118    Topic2   2.895325        love   4.572659   0.3879  -4.6008
0      Topic2   5.672121       great   9.844710   0.2935  -3.9283
2      Topic2   9.293196      camera  22.623762  -0.0448  -3.4346
37     Topic2   2.900399       canon   5.261331   0.2493  -4.5991
32     Topic2   3.736017        easy   7.256189   0.1810  -4.3459
89     Topic2   2.203823       price   3.943524   0.2630  -4.8737
56     Topic2   1.598936   different   2.617765   0.3519  -5.1946
4      Topic2   1.590753  everything   2.618475   0.3465  -5.1997
99     Topic2   2.850906     amazing   5.954735   0.1083  -4.6163
18     Topic2   2.274979      lenses   4.626464   0.1350  -4.8419
28     Topic2   2.241892         lot   4.629334   0.1198  -4.8566
20     Topic2   2.361224        good   5.997207  -0.0872  -4.8047
9      Topic2   2.228900         get   6.008684  -0.1468  -4.8624
53     Topic2   2.232576     quality  10.143035  -0.6688  -4.8607
280    Topic2   1.626056        came   3.304524   0.1357  -5.1777
66     Topic2   1.637301        nice   3.992661  -0.0465  -5.1708

In [17]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_unverified3 = pyLDAvis.gensim.prepare(lda_model_unverified3, doc_term_matrix_unverified ,dict_unverified)
vis_unverified3雌

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.093959 -0.007743       1        1  41.079183
1      0.055066 -0.063605       2        1  36.105224
0      0.038893  0.071348       3        1  22.815594, topic_info=     Category       Freq          Term      Total  loglift  logprob
term                                                               
130   Default   7.000000      pictures   7.000000  30.0000  30.0000
197   Default   4.000000         money   4.000000  29.0000  29.0000
226   Default   2.000000          gift   2.000000  28.0000  28.0000
116   Default   3.000000         spend   3.000000  27.0000  27.0000
18    Default   4.000000        lenses   4.000000  26.0000  26.0000
31    Default   3.000000           buy   3.000000  25.0000  25.0000
129   Default   3.000000         takes   3.000000  24.0000  24.0000
17    Default   3.000000           kit   3.000000  23.0000  23.0000
40    Default   3.000000          like   3.000000  22.0000  22.0000
2     Default  22.000000        camera  22.000000  21.0000  21.0000
233   Default   1.000000       picture   1.000000  20.0000  20.0000
227   Default   1.000000          card   1.000000  19.0000  19.0000
222   Default   1.000000       wanting   1.000000  18.0000  18.0000
225   Default   1.000000     advantage   1.000000  17.0000  17.0000
293   Default   1.000000       filming   1.000000  16.0000  16.0000
305   Default   1.000000          best   1.000000  15.0000  15.0000
324   Default   1.000000      friendly   1.000000  14.0000  14.0000
323   Default   1.000000          user   1.000000  13.0000  13.0000
145   Default   3.000000      settings   3.000000  12.0000  12.0000
205   Default   3.000000        really   3.000000  11.0000  11.0000
93    Default   2.000000       upgrade   2.000000  10.0000  10.0000
202   Default   2.000000       reading   2.000000   9.0000   9.0000
172   Default   2.000000          auto   2.000000   8.0000   8.0000
0     Default   9.000000         great   9.000000   7.0000   7.0000
104   Default   3.000000          well   3.000000   6.0000   6.0000
111   Default   3.000000       overall   3.000000   5.0000   5.0000
61    Default   6.000000          lens   6.000000   4.0000   4.0000
20    Default   6.000000          good   6.000000   3.0000   3.0000
118   Default   4.000000          love   4.000000   2.0000   2.0000
32    Default   7.000000          easy   7.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
278    Topic3   0.769988        unlock   1.233321   1.0066  -5.2924
273    Topic3   0.769988  professional   1.233321   1.0066  -5.2924
272    Topic3   0.769988         loves   1.233321   1.0066  -5.2924
277    Topic3   0.769988         truly   1.233321   1.0066  -5.2924
275    Topic3   0.769988          book   1.233321   1.0066  -5.2924
276    Topic3   0.769988         class   1.233321   1.0066  -5.2924
274    Topic3   0.769988        simple   1.233321   1.0066  -5.2924
279    Topic3   0.769988       feature   1.233321   1.0066  -5.2924
290    Topic3   0.769926         issue   1.233333   1.0065  -5.2925
116    Topic3   1.920621         spend   3.076686   1.0065  -4.3784
197    Topic3   2.371771         money   4.399108   0.8600  -4.1674
31     Topic3   1.919441           buy   3.764172   0.8042  -4.3790
129    Topic3   1.923879         takes   3.793520   0.7988  -4.3767
93     Topic3   1.349688       upgrade   2.497735   0.8622  -4.7312
202    Topic3   1.349470       reading   2.512873   0.8560  -4.7313
172    Topic3   1.348677          auto   2.513044   0.8554  -4.7319
2      Topic3   6.159709        camera  22.362050   0.1884  -3.2130
0      Topic3   3.089391         great   9.817285   0.3216  -3.9031
130    Topic3   2.496076      pictures   7.181881   0.4209  -4.1163
32     Topic3   1.916526          easy   7.231027   0.1499  -4.3805
7      Topic3   1.925763          take   7.931812   0.0622  -4.3757
37     Topic3  

In [18]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_unverified4 = pyLDAvis.gensim.prepare(lda_model_unverified4, doc_term_matrix_unverified ,dict_unverified)
vis_unverified4

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.104058 -0.048366       1        1  37.191877
2     -0.011144  0.089575       2        1  26.731272
0      0.075353 -0.067390       3        1  23.496010
1      0.039850  0.026181       4        1  12.580841, topic_info=     Category       Freq       Term      Total  loglift  logprob
term                                                            
226   Default   2.000000       gift   2.000000  30.0000  30.0000
100   Default   7.000000        use   7.000000  29.0000  29.0000
18    Default   5.000000     lenses   5.000000  28.0000  28.0000
93    Default   2.000000    upgrade   2.000000  27.0000  27.0000
172   Default   2.000000       auto   2.000000  26.0000  26.0000
129   Default   3.000000      takes   3.000000  25.0000  25.0000
99    Default   6.000000    amazing   6.000000  24.0000  24.0000
227   Default   1.000000       card   1.000000  23.0000  23.0000
233   Default   1.000000    picture   1.000000  22.0000  22.0000
225   Default   1.000000  advantage   1.000000  21.0000  21.0000
222   Default   1.000000    wanting   1.000000  20.0000  20.0000
324   Default   1.000000   friendly   1.000000  19.0000  19.0000
323   Default   1.000000       user   1.000000  18.0000  18.0000
147   Default   2.000000     taking   2.000000  17.0000  17.0000
121   Default   2.000000      using   2.000000  16.0000  16.0000
130   Default   7.000000   pictures   7.000000  15.0000  15.0000
140   Default   4.000000      would   4.000000  14.0000  14.0000
52    Default   4.000000     better   4.000000  13.0000  13.0000
187   Default   2.000000     little   2.000000  12.0000  12.0000
53    Default  10.000000    quality  10.000000  11.0000  11.0000
7     Default   7.000000       take   7.000000  10.0000  10.0000
101   Default   2.000000       much   2.000000   9.0000   9.0000
61    Default   6.000000       lens   6.000000   8.0000   8.0000
280   Default   3.000000       came   3.000000   7.0000   7.0000
20    Default   6.000000       good   6.000000   6.0000   6.0000
38    Default   3.000000      point   3.000000   5.0000   5.0000
39    Default   3.000000      shoot   3.000000   4.0000   4.0000
106   Default   3.000000        got   3.000000   3.0000   3.0000
145   Default   3.000000   settings   3.000000   2.0000   2.0000
253   Default   1.000000    another   1.000000   1.0000   1.0000
...       ...        ...        ...        ...      ...      ...
302    Topic4   0.636414     become   1.153794   1.4780  -4.8877
296    Topic4   0.636414      black   1.153794   1.4780  -4.8877
322    Topic4   0.636405      taken   1.153797   1.4780  -4.8877
325    Topic4   0.636405        top   1.153797   1.4780  -4.8877
321    Topic4   0.636405   thrilled   1.153797   1.4780  -4.8877
320    Topic4   0.636405     artist   1.153797   1.4780  -4.8877
319    Topic4   0.636405      local   1.153797   1.4780  -4.8877
93     Topic4   1.146243    upgrade   2.345340   1.3571  -4.2993
172    Topic4   1.145927       auto   2.345448   1.3567  -4.2996
129    Topic4   1.151924      takes   3.647304   0.9204  -4.2943
100    Topic4   1.657809        use   7.561221   0.5555  -3.9303
130    Topic4   1.146416   pictures   7.083388   0.2519  -4.2991
7      Topic4   1.145927       take   7.849573   0.1487  -4.2996
0      Topic4   1.147809      great  10.126880  -0.1043  -4.2979
2      Topic4   1.147160     camera  23.744568  -0.9571  -4.2985
116    Topic4   0.639355      spend   3.221776   0.4558  -4.8831
106    Topic4   0.639291        got   3.170659   0.4717  -4.8832
197    Topic4   0.638951      money   4.691714   0.0793  -4.8837
20     Topic4   0.638295       good   6.131431  -0.1894  -4.8847
300    Topic4   0.637877     trying   1.887854   0.9879  -4.8854
202    Topic4   0.637586    reading   2.570601   0.6788  -4.8858
224    Topic4   0.637074       took   1.888214   0.9865  -4.8866
235    Topic4   0.636913        far   1.888286   0.9862  -

In [19]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_unverified5 = pyLDAvis.gensim.prepare(lda_model_unverified5, doc_term_matrix_unverified ,dict_unverified)
vis_unverified5

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.031944  0.114915       1        1  39.210173
2     -0.111952 -0.044507       2        1  32.432831
0      0.102130 -0.083768       3        1  18.036075
3     -0.024671  0.004860       4        1   8.089284
4      0.002549  0.008500       5        1   2.231638, topic_info=     Category       Freq        Term      Total  loglift  logprob
term                                                             
0     Default   9.000000       great   9.000000  30.0000  30.0000
2     Default  23.000000      camera  23.000000  29.0000  29.0000
99    Default   6.000000     amazing   6.000000  28.0000  28.0000
197   Default   4.000000       money   4.000000  27.0000  27.0000
104   Default   2.000000        well   2.000000  26.0000  26.0000
226   Default   2.000000        gift   2.000000  25.0000  25.0000
20    Default   5.000000        good   5.000000  24.0000  24.0000
129   Default   3.000000       takes   3.000000  23.0000  23.0000
31    Default   3.000000         buy   3.000000  22.0000  22.0000
53    Default  10.000000     quality  10.000000  21.0000  21.0000
130   Default   7.000000    pictures   7.000000  20.0000  20.0000
323   Default   1.000000        user   1.000000  19.0000  19.0000
324   Default   1.000000    friendly   1.000000  18.0000  18.0000
263   Default   1.000000   excellent   1.000000  17.0000  17.0000
5     Default   3.000000        need   3.000000  16.0000  16.0000
4     Default   2.000000  everything   2.000000  15.0000  15.0000
52    Default   4.000000      better   4.000000  14.0000  14.0000
147   Default   2.000000      taking   2.000000  13.0000  13.0000
119   Default   4.000000       first   4.000000  12.0000  12.0000
205   Default   3.000000      really   3.000000  11.0000  11.0000
34    Default   3.000000       learn   3.000000  10.0000  10.0000
337   Default   0.000000      nature   0.000000   9.0000   9.0000
66    Default   3.000000        nice   3.000000   8.0000   8.0000
140   Default   4.000000       would   4.000000   7.0000   7.0000
139   Default   1.000000        loud   1.000000   6.0000   6.0000
126   Default   1.000000     finally   1.000000   5.0000   5.0000
136   Default   1.000000      mirror   1.000000   4.0000   4.0000
293   Default   1.000000     filming   1.000000   3.0000   3.0000
133   Default   1.000000      people   1.000000   2.0000   2.0000
264   Default   1.000000       wants   1.000000   1.0000   1.0000
...       ...        ...         ...        ...      ...      ...
272    Topic5   0.038004       loves   1.014741   0.5177  -5.9764
295    Topic5   0.038005      behind   1.213752   0.3387  -5.9764
292    Topic5   0.038005       ideal   1.213752   0.3387  -5.9764
294    Topic5   0.038005     started   1.213752   0.3387  -5.9764
291    Topic5   0.038005       built   1.213752   0.3387  -5.9764
290    Topic5   0.038005       issue   1.213752   0.3387  -5.9764
148    Topic5   0.038004      change   1.213754   0.3387  -5.9764
131    Topic5   0.038004       quiet   1.213754   0.3387  -5.9764
2      Topic5   0.225460      camera  23.978301  -0.8643  -4.1959
251    Topic5   0.038005      owning   1.213865   0.3386  -5.9764
330    Topic5   0.038013       still   2.084548  -0.2020  -5.9762
20     Topic5   0.038012        good   5.908603  -1.2438  -5.9762
89     Topic5   0.038011       price   4.293235  -0.9245  -5.9762
135    Topic5   0.038011       could   1.972858  -0.1469  -5.9762
106    Topic5   0.038011         got   3.202174  -0.6313  -5.9762
326    Topic5   0.038010   specially   1.345897   0.2355  -5.9762
333    Topic5   0.038010   beautiful   1.345900   0.2355  -5.9762
32     Topic5   0.038009        easy   7.801576  -1.5218  -5.9763
140    Topic5   0.038008       would   4.819378  -1.0402  -5.9763
119    Topic5   0.038008       first   4.784541  -1.0329  -5.9763
130    Topic5   0.038008    pictures   7.288438  -1.4538  -5.9763
66     Topic5  

In [40]:
unverifie_topics = lda_model_unverified2.get_document_topics(doc_term_matrix_unverified, per_word_topics=True)
review = 1
for doc_topics, review_word_topics, phi_values in unverifie_topics:
    print('review: {} \n'.format(review))
    print('review topics:', doc_topics)
    max = 0
    for i in range(len(doc_topics)):
        if max < doc_topics[i][1]:
            max = doc_topics[i][1]
            doc_topics_hightest = doc_topics[i][0]
            
    print('review topics(highest probs): Topic ', doc_topics_hightest)
    print("words' topics:", review_word_topics)
    print('words related to the topics:\n', lda_model_unverified2.print_topics(-1)[doc_topics_hightest][1])
    print('-------------- \n')
    review += 1

review: 1 

review topics: [(1, 0.9947169320869108)]
review topics(highest probs): Topic  1
words' topics: [(0, [1]), (1, [1]), (2, [1, 0]), (3, [1]), (4, [1]), (5, [1]), (6, [1]), (7, [1, 0]), (8, [1]), (9, [1]), (10, [1]), (11, [1]), (12, [1]), (13, [1]), (14, [1]), (15, [1]), (16, [1]), (17, [1]), (18, [1]), (19, [1]), (20, [1]), (21, [1]), (22, [1]), (23, [1]), (24, [1]), (25, [1]), (26, [1]), (27, [1]), (28, [1]), (29, [1]), (30, [1]), (31, [1]), (32, [1]), (33, [1]), (34, [1]), (35, [1]), (36, [1]), (37, [1]), (38, [1]), (39, [1]), (40, [1]), (41, [1]), (42, [1]), (43, [1]), (44, [1]), (45, [1]), (46, [1]), (47, [1]), (48, [1]), (49, [1]), (50, [1]), (51, [1]), (52, [1]), (53, [1, 0]), (54, [1]), (55, [1]), (56, [1]), (57, [1]), (58, [1]), (59, [1]), (60, [1]), (61, [1]), (62, [1]), (63, [1]), (64, [1]), (65, [1]), (66, [1]), (67, [1]), (68, [1]), (69, [1]), (70, [1]), (71, [1]), (72, [1]), (73, [1]), (74, [1]), (75, [1]), (76, [1]), (77, [1]), (78, [1]), (79, [1]), (80, [1]), (8

In [20]:
dict_verified = corpora.Dictionary(lda_list_verified)
print(dict_verified)
doc_term_matrix_verified = [dict_verified.doc2bow(doc) for doc in lda_list_verified]
# print(doc_term_matrix_verified)

Dictionary(2438 unique tokens: ['camera', 'full', 'featured', 'entry', 'level']...)


In [21]:
lda_model_verified3 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified, num_topics=3, id2word = dict_verified, passes=50)

In [22]:
lda_model_verified4 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified, num_topics=4, id2word = dict_verified, passes=50)

In [23]:
lda_model_verified5 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified, num_topics=5, id2word = dict_verified, passes=50)

In [24]:
lda_model_verified6 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified, num_topics=6, id2word = dict_verified, passes=50)

In [25]:
lda_model_verified7 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified, num_topics=7, id2word = dict_verified, passes=50)

In [26]:
lda_model_verified8 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified, num_topics=8, id2word = dict_verified, passes=50)

In [27]:
for idx_verified, topic_verified in lda_model_verified3.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.040*"camera" + 0.027*"canon" + 0.018*"easy" + 0.015*"pictures" + 0.014*"use" + 0.011*"phone" + 0.011*"rebel" + 0.010*"user" + 0.010*"used" + 0.009*"like"
Topic: 1 
Words: 0.069*"camera" + 0.044*"good" + 0.021*"would" + 0.015*"one" + 0.015*"pictures" + 0.015*"take" + 0.014*"bought" + 0.013*"great" + 0.013*"recommend" + 0.013*"still"
Topic: 2 
Words: 0.117*"great" + 0.079*"camera" + 0.061*"easy" + 0.042*"love" + 0.037*"use" + 0.036*"takes" + 0.024*"pictures" + 0.018*"quality" + 0.016*"loves" + 0.015*"bought"


In [28]:
for idx_verified, topic_verified in lda_model_verified4.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0.049*"use" + 0.045*"takes" + 0.044*"pictures" + 0.027*"quality" + 0.019*"good" + 0.017*"beginner"
Topic: 1 
Words: 0.060*"camera" + 0.022*"would" + 0.018*"like" + 0.016*"recommend" + 0.016*"good" + 0.015*"pictures" + 0.014*"canon" + 0.013*"using" + 0.012*"used" + 0.011*"use"
Topic: 2 
Words: 0.070*"camera" + 0.068*"great" + 0.026*"lens" + 0.025*"price" + 0.016*"get" + 0.015*"good" + 0.015*"bundle" + 0.014*"lenses" + 0.013*"came" + 0.013*"package"
Topic: 3 
Words: 0.056*"bought" + 0.052*"camera" + 0.041*"loves" + 0.039*"best" + 0.033*"daughter" + 0.029*"got" + 0.027*"gift" + 0.024*"great" + 0.021*"photography" + 0.021*"buy"


In [29]:
for idx_verified, topic_verified in lda_model_verified5.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.100*"great" + 0.081*"easy" + 0.078*"camera" + 0.046*"love" + 0.045*"use" + 0.040*"takes" + 0.028*"pictures" + 0.023*"good" + 0.023*"quality" + 0.013*"really"
Topic: 1 
Words: 0.069*"camera" + 0.034*"great" + 0.024*"good" + 0.024*"would" + 0.021*"recommend" + 0.015*"get" + 0.012*"buy" + 0.011*"best" + 0.010*"price" + 0.010*"lens"
Topic: 2 
Words: 0.055*"canon" + 0.047*"best" + 0.044*"camera" + 0.026*"one" + 0.025*"rebel" + 0.024*"happy" + 0.020*"new" + 0.019*"ever" + 0.018*"great" + 0.017*"cameras"
Topic: 3 
Words: 0.074*"pictures" + 0.047*"camera" + 0.038*"taking" + 0.032*"take" + 0.020*"took" + 0.018*"using" + 0.016*"used" + 0.015*"phone" + 0.014*"family" + 0.013*"bought"
Topic: 4 
Words: 0.058*"bought" + 0.055*"camera" + 0.048*"loves" + 0.036*"daughter" + 0.035*"great" + 0.031*"gift" + 0.029*"got" + 0.023*"wife" + 0.018*"purchased" + 0.017*"one"


In [30]:
for idx_verified, topic_verified in lda_model_verified6.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.052*"camera" + 0.040*"good" + 0.020*"easy" + 0.017*"use" + 0.015*"like" + 0.014*"lens" + 0.011*"still" + 0.010*"used" + 0.010*"really" + 0.010*"canon"
Topic: 1 
Words: 0.048*"pictures" + 0.030*"camera" + 0.027*"get" + 0.022*"taking" + 0.016*"really" + 0.015*"love" + 0.015*"come" + 0.014*"take" + 0.013*"quality" + 0.011*"wait"
Topic: 2 
Words: 0.120*"great" + 0.084*"easy" + 0.077*"camera" + 0.053*"use" + 0.047*"takes" + 0.035*"love" + 0.032*"pictures" + 0.023*"quality" + 0.013*"learn" + 0.012*"first"
Topic: 3 
Words: 0.083*"bought" + 0.073*"loves" + 0.058*"daughter" + 0.050*"camera" + 0.035*"wife" + 0.033*"gift" + 0.022*"photography" + 0.021*"son" + 0.019*"happy" + 0.019*"old"
Topic: 4 
Words: 0.061*"camera" + 0.028*"pictures" + 0.027*"good" + 0.025*"take" + 0.023*"better" + 0.022*"worth" + 0.019*"quality" + 0.019*"using" + 0.014*"product" + 0.013*"every"
Topic: 5 
Words: 0.086*"camera" + 0.075*"great" + 0.028*"would" + 0.025*"recommend" + 0.024*"best" + 0.023*"love" 

In [31]:
for idx_verified, topic_verified in lda_model_verified7.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.070*"great" + 0.049*"camera" + 0.031*"everything" + 0.025*"taking" + 0.020*"photos" + 0.020*"need" + 0.020*"get" + 0.016*"needed" + 0.014*"shots" + 0.014*"package"
Topic: 1 
Words: 0.135*"great" + 0.093*"easy" + 0.085*"camera" + 0.059*"takes" + 0.058*"use" + 0.033*"pictures" + 0.030*"quality" + 0.023*"good" + 0.019*"beginner" + 0.011*"awesome"
Topic: 2 
Words: 0.062*"camera" + 0.054*"love" + 0.034*"pictures" + 0.033*"easy" + 0.027*"great" + 0.025*"use" + 0.024*"take" + 0.018*"still" + 0.012*"learning" + 0.012*"canon"
Topic: 3 
Words: 0.057*"camera" + 0.030*"like" + 0.028*"good" + 0.020*"using" + 0.019*"used" + 0.017*"features" + 0.017*"get" + 0.014*"enjoyed" + 0.013*"digital" + 0.013*"cameras"
Topic: 4 
Words: 0.074*"lens" + 0.036*"camera" + 0.034*"lenses" + 0.032*"two" + 0.029*"extra" + 0.029*"came" + 0.022*"kit" + 0.021*"bag" + 0.021*"comes" + 0.015*"like"
Topic: 5 
Words: 0.065*"camera" + 0.044*"would" + 0.044*"good" + 0.040*"recommend" + 0.037*"great" + 0.036*"be

In [32]:
for idx_verified, topic_verified in lda_model_verified8.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.096*"love" + 0.076*"camera" + 0.055*"great" + 0.050*"recommend" + 0.044*"would" + 0.041*"first" + 0.036*"canon" + 0.023*"time" + 0.021*"awesome" + 0.017*"absolutely"
Topic: 1 
Words: 0.154*"easy" + 0.115*"great" + 0.090*"use" + 0.071*"takes" + 0.070*"camera" + 0.050*"pictures" + 0.034*"quality" + 0.030*"love" + 0.022*"good" + 0.019*"amazing"
Topic: 2 
Words: 0.065*"camera" + 0.054*"great" + 0.036*"get" + 0.017*"need" + 0.016*"everything" + 0.015*"shots" + 0.014*"lens" + 0.009*"quality" + 0.009*"want" + 0.009*"lenses"
Topic: 3 
Words: 0.052*"camera" + 0.023*"pictures" + 0.016*"take" + 0.014*"canon" + 0.014*"used" + 0.013*"phone" + 0.012*"one" + 0.011*"get" + 0.011*"like" + 0.011*"cameras"
Topic: 4 
Words: 0.090*"great" + 0.060*"camera" + 0.047*"price" + 0.037*"lens" + 0.031*"came" + 0.029*"bundle" + 0.028*"deal" + 0.022*"package" + 0.022*"got" + 0.021*"buy"
Topic: 5 
Words: 0.107*"nice" + 0.026*"camera" + 0.022*"amazing" + 0.022*"functions" + 0.018*"happy" + 0.015*"mo

In [33]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified3 = pyLDAvis.gensim.prepare(lda_model_verified3, doc_term_matrix_verified ,dict_verified)
vis_verified3

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.026018 -0.170384       1        1  37.242376
2     -0.185327  0.065898       2        1  36.278382
0      0.159309  0.104487       3        1  26.479242, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
93    Default  1620.000000        great  1620.000000  30.0000  30.0000
66    Default   555.000000         good   555.000000  29.0000  29.0000
86    Default   901.000000         easy   901.000000  28.0000  28.0000
410   Default   436.000000        takes   436.000000  27.0000  27.0000
405   Default   564.000000         love   564.000000  26.0000  26.0000
135   Default   245.000000        canon   245.000000  25.0000  25.0000
404   Default   258.000000        would   258.000000  24.0000  24.0000
134   Default   603.000000          use   603.000000  23.0000  23.0000
1008  Default   196.000000        loves   196.000000  22.0000  22.0000
429   Default   163.000000        still   163.000000  21.0000  21.0000
11    Default   160.000000      looking   160.000000  20.0000  20.0000
295   Default   166.000000  photography   166.000000  19.0000  19.0000
209   Default    98.000000        phone    98.000000  18.0000  18.0000
231   Default    95.000000        rebel    95.000000  17.0000  17.0000
8     Default   217.000000          one   217.000000  16.0000  16.0000
268   Default   130.000000     learning   130.000000  15.0000  15.0000
234   Default    90.000000         user    90.000000  14.0000  14.0000
334   Default   165.000000       taking   165.000000  13.0000  13.0000
515   Default   245.000000         take   245.000000  12.0000  12.0000
301   Default   360.000000       bought   360.000000  11.0000  11.0000
509   Default   151.000000      amazing   151.000000  10.0000  10.0000
257   Default   175.000000     beginner   175.000000   9.0000   9.0000
79    Default   246.000000          get   246.000000   8.0000   8.0000
511   Default   222.000000    recommend   222.000000   7.0000   7.0000
164   Default   138.000000         used   138.000000   6.0000   6.0000
599   Default   125.000000      awesome   125.000000   5.0000   5.0000
14    Default    62.000000        shoot    62.000000   4.0000   4.0000
587   Default    84.000000        start    84.000000   3.0000   3.0000
775   Default    81.000000   absolutely    81.000000   2.0000   2.0000
438   Default   230.000000        price   230.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
262    Topic3    23.298268       allows    24.015333   1.2985  -5.9481
276    Topic3    23.797722         find    24.540652   1.2981  -5.9269
171    Topic3    22.770112          day    23.487662   1.2978  -5.9710
260    Topic3    22.408415      setting    23.136892   1.2968  -5.9870
922    Topic3    20.838929    extremely    21.557243   1.2949  -6.0597
1225   Topic3    19.445563       sports    20.153897   1.2930  -6.1289
705    Topic3    19.775598        brand    20.504252   1.2926  -6.1120
510    Topic3    19.754919       option    20.500347   1.2918  -6.1131
34     Topic3    38.869898        years    42.963244   1.2287  -5.4363
59     Topic3    57.728667     settings    68.239282   1.1615  -5.0407
168    Topic3    56.291538         able    72.994350   1.0690  -5.0659
569    Topic3    34.366968      options    40.472656   1.1653  -5.5594
164    Topic3    86.464724         used   138.479441   0.8578  -4.6367
106    Topic3    42.248087       little    58.810988   0.9980  -5.3529
0      Topic3   354.265067       camera  2181.422718  -0.4889  -3.2264
64     Topic3    70.110866       lenses   147.216036   0.5870  -4.8464
187    Topic3    84.418267         like   232.572077   0.3154  -4.6607
98     Topic3    44.009278        shots    71.539259   0.8430  -5.3121
30     Topic3    62.126117        using   139.729089   0.5183  -4.

In [34]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified4 = pyLDAvis.gensim.prepare(lda_model_verified4, doc_term_matrix_verified ,dict_verified)
vis_verified4

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.053854  0.007981       1        1  33.532340
0     -0.210512  0.071037       2        1  28.176092
2      0.046927 -0.190989       3        1  23.189073
3      0.217439  0.111970       4        1  15.102495, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
86    Default   876.000000         easy   876.000000  30.0000  30.0000
93    Default  1650.000000        great  1650.000000  29.0000  29.0000
410   Default   430.000000        takes   430.000000  28.0000  28.0000
301   Default   371.000000       bought   371.000000  27.0000  27.0000
405   Default   561.000000         love   561.000000  26.0000  26.0000
134   Default   595.000000          use   595.000000  25.0000  25.0000
1008  Default   208.000000        loves   208.000000  24.0000  24.0000
143   Default   593.000000     pictures   593.000000  23.0000  23.0000
223   Default   258.000000         best   258.000000  22.0000  22.0000
701   Default   166.000000     daughter   166.000000  21.0000  21.0000
72    Default   201.000000         lens   201.000000  20.0000  20.0000
642   Default   174.000000          got   174.000000  19.0000  19.0000
693   Default   137.000000         gift   137.000000  18.0000  18.0000
404   Default   269.000000        would   269.000000  17.0000  17.0000
438   Default   246.000000        price   246.000000  16.0000  16.0000
315   Default   150.000000          buy   150.000000  15.0000  15.0000
1007  Default    99.000000         wife    99.000000  14.0000  14.0000
121   Default   366.000000      quality   366.000000  13.0000  13.0000
295   Default   164.000000  photography   164.000000  12.0000  12.0000
257   Default   171.000000     beginner   171.000000  11.0000  11.0000
538   Default   116.000000       bundle   116.000000  10.0000  10.0000
514   Default    85.000000        loved    85.000000   9.0000   9.0000
187   Default   241.000000         like   241.000000   8.0000   8.0000
30    Default   142.000000        using   142.000000   7.0000   7.0000
164   Default   140.000000         used   140.000000   6.0000   6.0000
79    Default   248.000000          get   248.000000   5.0000   5.0000
509   Default   150.000000      amazing   150.000000   4.0000   4.0000
511   Default   231.000000    recommend   231.000000   3.0000   3.0000
339   Default   129.000000         came   129.000000   2.0000   2.0000
786   Default   117.000000      package   117.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
586    Topic4    17.281775       thanks    19.167281   1.7868  -5.6853
1330   Topic4     6.783271          ask     7.570232   1.7805  -6.6205
766    Topic4     6.462735      offered     7.251531   1.7752  -6.6689
735    Topic4     5.987006         seen     6.771830   1.7671  -6.7454
1237   Topic4     5.833259        young     6.611588   1.7651  -6.7714
642    Topic4   148.037076          got   174.694840   1.7247  -3.5375
301    Topic4   283.877033       bought   371.801049   1.6205  -2.8864
223    Topic4   199.413135         best   258.897750   1.6293  -3.2396
315    Topic4   105.036487          buy   150.860298   1.5283  -3.8807
295    Topic4   106.638463  photography   164.463572   1.4571  -3.8655
406    Topic4    10.469269     problems    12.289004   1.7300  -6.1865
230    Topic4    73.047603    purchased   137.511881   1.2577  -4.2439
649    Topic4    23.316819        class    33.685179   1.5224  -5.3858
650    Topic4    19.300208       school    27.872136   1.5228  -5.5749
350    Topic4    37.357661          old    68.968221   1.2772  -4.9144
775    Topic4    37.766622   absolutely    82.709835   1.1064  -4.9035
409    Topic4    49.421791        happy   130.024020   0.9230  -4.6346
8      Topic4    66.823764          one   217.467370   0.7103  -4.3329
0      Topic

In [35]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified5 = pyLDAvis.gensim.prepare(lda_model_verified5, doc_term_matrix_verified ,dict_verified)
vis_verified5

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.018087 -0.002527       1        1  33.319765
0     -0.224914  0.005683       2        1  31.608573
4      0.081604 -0.231593       3        1  13.056168
3     -0.045817  0.107512       4        1  12.645820
2      0.207213  0.120925       5        1   9.369674, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
86    Default   867.000000         easy   867.000000  30.0000  30.0000
143   Default   609.000000     pictures   609.000000  29.0000  29.0000
410   Default   430.000000        takes   430.000000  28.0000  28.0000
301   Default   377.000000       bought   377.000000  27.0000  27.0000
135   Default   236.000000        canon   236.000000  26.0000  26.0000
405   Default   556.000000         love   556.000000  25.0000  25.0000
223   Default   272.000000         best   272.000000  24.0000  24.0000
1008  Default   211.000000        loves   211.000000  23.0000  23.0000
93    Default  1652.000000        great  1652.000000  22.0000  22.0000
334   Default   171.000000       taking   171.000000  21.0000  21.0000
134   Default   590.000000          use   590.000000  20.0000  20.0000
701   Default   169.000000     daughter   169.000000  19.0000  19.0000
404   Default   268.000000        would   268.000000  18.0000  18.0000
693   Default   139.000000         gift   139.000000  17.0000  17.0000
642   Default   179.000000          got   179.000000  16.0000  16.0000
8     Default   216.000000          one   216.000000  15.0000  15.0000
511   Default   236.000000    recommend   236.000000  14.0000  14.0000
515   Default   242.000000         take   242.000000  13.0000  13.0000
1007  Default   100.000000         wife   100.000000  12.0000  12.0000
231   Default    91.000000        rebel    91.000000  11.0000  11.0000
197   Default    88.000000         took    88.000000  10.0000  10.0000
409   Default   132.000000        happy   132.000000   9.0000   9.0000
9     Default   133.000000          new   133.000000   8.0000   8.0000
315   Default   160.000000          buy   160.000000   7.0000   7.0000
66    Default   536.000000         good   536.000000   6.0000   6.0000
1072  Default    59.000000         ever    59.000000   5.0000   5.0000
10    Default    89.000000      cameras    89.000000   4.0000   4.0000
230   Default   140.000000    purchased   140.000000   3.0000   3.0000
164   Default   137.000000         used   137.000000   2.0000   2.0000
30    Default   140.000000        using   140.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
1025   Topic5    11.037323  complicated    13.113434   2.1953  -5.6563
1379   Topic5     4.387444       nephew     5.215346   2.1948  -6.5788
90     Topic5     4.378249        delay     5.215656   2.1927  -6.5809
1090   Topic5    13.990737     memories    16.866653   2.1807  -5.4192
123    Topic5    46.581684       always    59.783017   2.1182  -4.2164
135    Topic5   173.355613        canon   236.257267   2.0581  -2.9022
180    Topic5    35.009203        never    50.676757   1.9978  -4.5020
223    Topic5   149.130303         best   272.059498   1.7665  -3.0528
827    Topic5    13.734528       picked    18.388192   2.0759  -5.4377
10     Topic5    53.934593      cameras    89.217738   1.8644  -4.0698
409    Topic5    74.683309        happy   132.871483   1.7916  -3.7443
589    Topic5    19.937498  recommended    28.350659   2.0156  -5.0650
9      Topic5    61.955356          new   133.679481   1.5987  -3.9312
166    Topic5    24.577133      upgrade    41.532912   1.8430  -4.8558
8      Topic5    81.732920          one   216.599959   1.3931  -3.6541
120    Topic5    41.745237     purchase    88.312580   1.6184  -4.3260
626    Topic5    15.942638   experience    24.696682   1.9300  -5.2886
0      Topic5   140.307355   

In [36]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified6 = pyLDAvis.gensim.prepare(lda_model_verified6, doc_term_matrix_verified ,dict_verified)
vis_verified6

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.122594 -0.168213       1        1  26.309828
0     -0.068241  0.038491       2        1  23.139417
5     -0.017191 -0.135991       3        1  22.732722
4     -0.055382  0.147771       4        1   9.960755
1     -0.049599  0.135315       5        1   9.379902
3      0.313007 -0.017373       6        1   8.477378, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
93    Default  1646.000000        great  1646.000000  30.0000  30.0000
86    Default   904.000000         easy   904.000000  29.0000  29.0000
301   Default   371.000000       bought   371.000000  28.0000  28.0000
1008  Default   210.000000        loves   210.000000  27.0000  27.0000
410   Default   440.000000        takes   440.000000  26.0000  26.0000
134   Default   604.000000          use   604.000000  25.0000  25.0000
701   Default   167.000000     daughter   167.000000  24.0000  24.0000
143   Default   614.000000     pictures   614.000000  23.0000  23.0000
66    Default   563.000000         good   563.000000  22.0000  22.0000
693   Default   138.000000         gift   138.000000  21.0000  21.0000
1007  Default   100.000000         wife   100.000000  20.0000  20.0000
511   Default   221.000000    recommend   221.000000  19.0000  19.0000
404   Default   251.000000        would   251.000000  18.0000  18.0000
295   Default   162.000000  photography   162.000000  17.0000  17.0000
79    Default   244.000000          get   244.000000  16.0000  16.0000
121   Default   376.000000      quality   376.000000  15.0000  15.0000
315   Default   149.000000          buy   149.000000  14.0000  14.0000
334   Default   167.000000       taking   167.000000  13.0000  13.0000
515   Default   243.000000         take   243.000000  12.0000  12.0000
223   Default   256.000000         best   256.000000  11.0000  11.0000
75    Default   132.000000       better   132.000000  10.0000  10.0000
405   Default   580.000000         love   580.000000   9.0000   9.0000
536   Default    74.000000        worth    74.000000   8.0000   8.0000
1243  Default    61.000000          son    61.000000   7.0000   7.0000
409   Default   129.000000        happy   129.000000   6.0000   6.0000
350   Default    68.000000          old    68.000000   5.0000   5.0000
11    Default   158.000000      looking   158.000000   4.0000   4.0000
30    Default   139.000000        using   139.000000   3.0000   3.0000
0     Default  2189.000000       camera  2189.000000   2.0000   2.0000
230   Default   135.000000    purchased   135.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
580    Topic6     3.875670       canons     4.741458   2.2661  -6.6028
1995   Topic6     3.551659        sound     4.418825   2.2493  -6.6901
905    Topic6     3.394157  replacement     4.257317   2.2412  -6.7355
1184   Topic6     3.392470     allowing     4.257541   2.2406  -6.7360
646    Topic6     3.390711         sell     4.257634   2.2401  -6.7365
350    Topic6    54.422323          old    68.573798   2.2366  -3.9607
450    Topic6    19.552956      several    25.687890   2.1949  -4.9844
1012   Topic6     6.900929        setup     8.819118   2.2225  -6.0259
693    Topic6    95.331705         gift   138.137479   2.0969  -3.4002
301    Topic6   236.820490       bought   371.631418   2.0172  -2.4902
303    Topic6    17.905831        older    24.888316   2.1385  -5.0724
1029   Topic6    15.165161      husband    22.298321   2.0823  -5.2385
543    Topic6    12.071522        model    18.476440   2.0421  -5.4667
650    Topic6    16.236064       school    27.934615   1.9251  -5.1703
409    Topic6    54.452603        happy   129.705254   1.5998  -3.9602
295    Topic6    62.964313  photography   162.610510   1.5190  -3.8149
460    Topic6     8.073995       course    11.

In [37]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified7 = pyLDAvis.gensim.prepare(lda_model_verified7, doc_term_matrix_verified ,dict_verified)
vis_verified7

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.065837 -0.080417       1        1  25.455214
1      0.205044 -0.122874       2        1  19.675330
5      0.058441  0.070924       3        1  16.083995
6      0.031251  0.252674       4        1  13.032889
0      0.008542 -0.019461       5        1   9.682309
3     -0.173157 -0.050328       6        1   8.319927
4     -0.195958 -0.050518       7        1   7.750337, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
86    Default   897.000000         easy   897.000000  30.0000  30.0000
93    Default  1650.000000        great  1650.000000  29.0000  29.0000
410   Default   432.000000        takes   432.000000  28.0000  28.0000
301   Default   371.000000       bought   371.000000  27.0000  27.0000
134   Default   599.000000          use   599.000000  26.0000  26.0000
72    Default   208.000000         lens   208.000000  25.0000  25.0000
1008  Default   212.000000        loves   212.000000  24.0000  24.0000
404   Default   248.000000        would   248.000000  23.0000  23.0000
511   Default   219.000000    recommend   219.000000  22.0000  22.0000
405   Default   619.000000         love   619.000000  21.0000  21.0000
701   Default   169.000000     daughter   169.000000  20.0000  20.0000
66    Default   529.000000         good   529.000000  19.0000  19.0000
64    Default   153.000000       lenses   153.000000  18.0000  18.0000
315   Default   148.000000          buy   148.000000  17.0000  17.0000
693   Default   140.000000         gift   140.000000  16.0000  16.0000
143   Default   610.000000     pictures   610.000000  15.0000  15.0000
187   Default   235.000000         like   235.000000  14.0000  14.0000
223   Default   257.000000         best   257.000000  13.0000  13.0000
515   Default   245.000000         take   245.000000  12.0000  12.0000
377   Default   112.000000   everything   112.000000  11.0000  11.0000
349   Default    83.000000          two    83.000000  10.0000  10.0000
334   Default   166.000000       taking   166.000000   9.0000   9.0000
1007  Default   101.000000         wife   101.000000   8.0000   8.0000
591   Default    78.000000        extra    78.000000   7.0000   7.0000
339   Default   130.000000         came   130.000000   6.0000   6.0000
233   Default   108.000000      product   108.000000   5.0000   5.0000
8     Default   210.000000          one   210.000000   4.0000   4.0000
429   Default   166.000000        still   166.000000   3.0000   3.0000
438   Default   232.000000        price   232.000000   2.0000   2.0000
121   Default   365.000000      quality   365.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
1268   Topic7     4.984190        shown     5.861720   2.3953  -6.2616
928    Topic7     4.636385       member     5.516419   2.3836  -6.3339
361    Topic7    10.365048         fast    12.395329   2.3786  -5.5294
325    Topic7     4.466140  flexibility     5.344277   2.3779  -6.3713
750    Topic7     4.465918        might     5.344264   2.3779  -6.3714
1448   Topic7     4.465516        stuff     5.344213   2.3778  -6.3715
281    Topic7     8.629492     standard    10.376300   2.3731  -5.7127
1427   Topic7     3.776378         star     4.653934   2.3485  -6.5391
114    Topic7    58.562918          kit    72.280769   2.3470  -3.7977
383    Topic7    14.285597       couple    18.108963   2.3203  -5.2086
389    Topic7    54.112935        comes    74.909221   2.2322  -3.8768
195    Topic7    16.820218          say    23.108083   2.2398  -5.0453
64     Topic7    89.547702       lenses   153.999551   2.0153  -3.3731
339    Topic7    76.999365         came   130.133582   2.0327  -3.5241
239    Topic7    16.620202       choice    24.444873   2.1716  -5.0572
63     Topic7    15.425628     included    23.972568   2.1166  

In [38]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified8 = pyLDAvis.gensim.prepare(lda_model_verified8, doc_term_matrix_verified ,dict_verified)
vis_verified8

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.080205  0.079277       1        1  22.332035
1      0.261142  0.063662       2        1  16.351228
7      0.073762  0.007809       3        1  13.719434
2     -0.024869  0.088217       4        1  11.793195
4      0.033928 -0.076152       5        1  11.357041
6     -0.127294 -0.244837       6        1  10.774418
0      0.083906 -0.062954       7        1   9.904399
5     -0.220369  0.144979       8        1   3.768250, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
86    Default   851.000000           easy   851.000000  30.0000  30.0000
134   Default   580.000000            use   580.000000  29.0000  29.0000
405   Default   566.000000           love   566.000000  28.0000  28.0000
410   Default   424.000000          takes   424.000000  27.0000  27.0000
301   Default   365.000000         bought   365.000000  26.0000  26.0000
93    Default  1654.000000          great  1654.000000  25.0000  25.0000
66    Default   528.000000           good   528.000000  24.0000  24.0000
665   Default   156.000000           nice   156.000000  23.0000  23.0000
1008  Default   208.000000          loves   208.000000  22.0000  22.0000
511   Default   214.000000      recommend   214.000000  21.0000  21.0000
701   Default   166.000000       daughter   166.000000  20.0000  20.0000
438   Default   238.000000          price   238.000000  19.0000  19.0000
404   Default   249.000000          would   249.000000  18.0000  18.0000
135   Default   231.000000          canon   231.000000  17.0000  17.0000
18    Default   210.000000          first   210.000000  16.0000  16.0000
143   Default   602.000000       pictures   602.000000  15.0000  15.0000
72    Default   197.000000           lens   197.000000  14.0000  14.0000
79    Default   252.000000            get   252.000000  13.0000  13.0000
693   Default   137.000000           gift   137.000000  12.0000  12.0000
515   Default   247.000000           take   247.000000  11.0000  11.0000
339   Default   123.000000           came   123.000000  10.0000  10.0000
538   Default   112.000000         bundle   112.000000   9.0000   9.0000
509   Default   152.000000        amazing   152.000000   8.0000   8.0000
408   Default   109.000000           deal   109.000000   7.0000   7.0000
1007  Default    99.000000           wife    99.000000   6.0000   6.0000
223   Default   262.000000           best   262.000000   5.0000   5.0000
642   Default   174.000000            got   174.000000   4.0000   4.0000
257   Default   178.000000       beginner   178.000000   3.0000   3.0000
599   Default   131.000000        awesome   131.000000   2.0000   2.0000
786   Default   114.000000        package   114.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
1388   Topic8     2.615176      upgrading     3.503605   2.9861  -6.1854
1497   Topic8     2.615126        selling     3.503597   2.9861  -6.1854
434    Topic8     2.615136         double     3.503616   2.9861  -6.1854
1452   Topic8     4.669474          niece     6.456253   2.9546  -5.6057
1460   Topic8     2.283687      promotion     3.172042   2.9500  -6.3209
1861   Topic8     2.283472       operated     3.172036   2.9499  -6.3210
435    Topic8    13.222010         decent    18.505154   2.9424  -4.5648
1285   Topic8     4.364150          items     6.125551   2.9395  -5.6733
589    Topic8    17.444941    recommended    27.554275   2.8215  -4.2877
939    Topic8    16.536616          liked    27.236707   2.7796  -4.3411
551    Topic8     8.538972  photographers    14.421255   2.7545  -5.0021
922    Topic8    10.247080      extremely    21.116290   2.5555  -4.8197
141    Topic8     6.344750       addition    11.326470   2.6990  -5.2991
1695   Topic8     5.593451           sons    10.173945   2.

In [42]:
verified_topics = lda_model_verified4.get_document_topics(doc_term_matrix_verified, per_word_topics=True)
review = 1
for doc_topics, review_word_topics, phi_values in verified_topics:
    print('review: {} \n'.format(review))
    print('review topics:', doc_topics)
    max = 0
    for i in range(len(doc_topics)):
        if max < doc_topics[i][1]:
            max = doc_topics[i][1]
            doc_topics_hightest = doc_topics[i][0]

    print('review topics(highest probs): Topic ', doc_topics_hightest)
    print('words related to the topics:\n', lda_model_verified4.print_topics(-1)[doc_topics_hightest][1])
    print("words' topics:", review_word_topics)
    print('-------------- \n')
    review += 1

review: 1 

review topics: [(0, 0.03333918420037379), (1, 0.33484499498778403), (2, 0.6307746860260159)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.068*"great" + 0.026*"lens" + 0.025*"price" + 0.016*"get" + 0.015*"good" + 0.015*"bundle" + 0.014*"lenses" + 0.013*"came" + 0.013*"package"
words' topics: [(0, [2, 1, 0]), (1, [2, 1]), (2, [2, 1]), (3, [2]), (4, [2]), (5, [2, 1]), (6, [1]), (7, [1, 2]), (8, [2, 1]), (9, [1, 2, 0]), (10, [1, 2]), (11, [1, 2]), (12, [2]), (13, [1]), (14, [1, 0]), (15, [1, 2]), (16, [1, 2]), (17, [2, 1]), (18, [1, 0]), (19, [2, 0]), (20, [1, 2]), (21, [2, 1]), (22, [2]), (23, [2]), (24, [2, 1]), (25, [2]), (26, [2, 1]), (27, [2]), (28, [2, 1]), (29, [2, 1, 0]), (30, [1]), (31, [1, 0]), (32, [2]), (33, [2, 1, 0]), (34, [1]), (35, [1, 2]), (36, [2, 1]), (37, [1, 2]), (38, [2]), (39, [1, 0]), (40, [2, 1]), (41, [1]), (42, [2, 0, 1]), (43, [1, 2]), (44, [1, 0, 2]), (45, [1, 0, 2]), (46, [2, 1]), (47, [2, 1, 0]), (48, [2,

review: 26 

review topics: [(0, 0.6487139901893813), (1, 0.14146350369074315), (3, 0.2022676869739713)]
review topics(highest probs): Topic  0
words related to the topics:
 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0.049*"use" + 0.045*"takes" + 0.044*"pictures" + 0.027*"quality" + 0.019*"good" + 0.017*"beginner"
words' topics: [(0, [0, 3, 1]), (7, [1, 3]), (26, [0, 1, 2]), (81, [0, 3, 1]), (86, [0, 1]), (93, [0, 3]), (121, [0, 1]), (123, [3, 1]), (270, [0, 1]), (295, [3, 1]), (405, [0]), (457, [1, 0]), (522, [0, 1]), (547, [3]), (587, [0, 1, 3]), (694, [0, 3, 1]), (770, [3, 1]), (771, [0, 3]), (772, [3, 0]), (773, [0, 1]), (774, [1, 0]), (775, [0, 3]), (776, [0]), (777, [0]), (778, [0, 3]), (779, [0]), (780, [0, 3]), (781, [0, 1, 3]), (782, [0]), (783, [0, 3]), (784, [0, 3]), (785, [0, 3])]
-------------- 

review: 27 

review topics: [(1, 0.2305192303872437), (2, 0.5273600707891474), (3, 0.23621697720001017)]
review topics(highest probs): Topic  2
words related t

review: 53 

review topics: [(0, 0.6810708397234635), (1, 0.06340999281062429), (3, 0.24793064369321263)]
review topics(highest probs): Topic  0
words related to the topics:
 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0.049*"use" + 0.045*"takes" + 0.044*"pictures" + 0.027*"quality" + 0.019*"good" + 0.017*"beginner"
words' topics: [(0, [0, 3, 1]), (14, [0, 1]), (26, [0, 1, 2]), (66, [0, 3, 1]), (86, [0]), (116, [0, 1]), (121, [0, 1]), (129, [0, 3, 1]), (135, [0, 1, 3]), (143, [0, 1]), (192, [0, 1]), (223, [3, 0]), (268, [0, 3, 1]), (276, [0, 1]), (315, [3]), (350, [3, 1]), (519, [0, 3, 1]), (586, [3]), (602, [3]), (650, [3, 1]), (832, [0]), (985, [0]), (986, [0]), (987, [0]), (988, [0]), (989, [0, 3]), (990, [0, 1, 3]), (991, [0, 3]), (992, [0, 3]), (993, [0]), (994, [0]), (995, [0]), (996, [3, 1, 0])]
-------------- 

review: 54 

review topics: [(0, 0.22587058923703643), (1, 0.3837002380636973), (2, 0.34635400857451265), (3, 0.04407516412475378)]
review topics(high

words related to the topics:
 0.070*"camera" + 0.068*"great" + 0.026*"lens" + 0.025*"price" + 0.016*"get" + 0.015*"good" + 0.015*"bundle" + 0.014*"lenses" + 0.013*"came" + 0.013*"package"
words' topics: [(0, [2, 3]), (15, [2]), (72, [2]), (79, [2, 3]), (106, [2]), (131, [2, 3]), (295, [3, 2]), (296, [2]), (316, [3, 2]), (364, [2]), (376, [3, 2]), (402, [2]), (405, [2, 3]), (431, [2, 3]), (452, [2]), (485, [3, 2]), (538, [2]), (556, [3, 2]), (591, [2]), (602, [3]), (642, [3, 2]), (665, [2, 3]), (707, [3]), (746, [3]), (775, [3]), (1029, [3]), (1033, [2]), (1056, [2]), (1157, [3]), (1158, [2, 3]), (1159, [3, 2]), (1160, [2, 3]), (1161, [3])]
-------------- 

review: 86 

review topics: [(1, 0.4348188240134793), (2, 0.5517731978447947)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.068*"great" + 0.026*"lens" + 0.025*"price" + 0.016*"get" + 0.015*"good" + 0.015*"bundle" + 0.014*"lenses" + 0.013*"came" + 0.013*"package"
words' topics: [(0, [2, 1]), 

words related to the topics:
 0.060*"camera" + 0.022*"would" + 0.018*"like" + 0.016*"recommend" + 0.016*"good" + 0.015*"pictures" + 0.014*"canon" + 0.013*"using" + 0.012*"used" + 0.011*"use"
words' topics: [(0, [1, 0, 2]), (18, [1, 0]), (56, [1, 2]), (57, [0, 1, 2]), (86, [0, 1]), (93, [0, 2, 1]), (121, [0, 1, 2]), (134, [0, 1]), (158, [1, 2]), (268, [1, 0]), (369, [1]), (377, [2, 1]), (442, [1]), (509, [0, 1]), (530, [1]), (536, [1, 2]), (538, [2]), (582, [1, 2, 0]), (587, [1, 0]), (598, [1, 0]), (627, [0, 1, 2]), (668, [1]), (698, [1]), (828, [1, 2, 0]), (881, [1]), (1266, [1])]
-------------- 

review: 118 

review topics: [(0, 0.2506728523501609), (1, 0.3563760302921727), (2, 0.013637654166392833), (3, 0.37931346319127346)]
review topics(highest probs): Topic  3
words related to the topics:
 0.056*"bought" + 0.052*"camera" + 0.041*"loves" + 0.039*"best" + 0.033*"daughter" + 0.029*"got" + 0.027*"gift" + 0.024*"great" + 0.021*"photography" + 0.021*"buy"
words' topics: [(0, [1, 3, 0])

...

words related to the topics:
 0.056*"bought" + 0.052*"camera" + 0.041*"loves" + 0.039*"best" + 0.033*"daughter" + 0.029*"got" + 0.027*"gift" + 0.024*"great" + 0.021*"photography" + 0.021*"buy"
words' topics: [(171, [3]), (1002, [0])]
-------------- 

review: 3319 

review topics: [(0, 0.03719461803206232), (1, 0.03962180727467239), (2, 0.6318615504162317), (3, 0.2913220242770335)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.068*"great" + 0.026*"lens" + 0.025*"price" + 0.016*"get" + 0.015*"good" + 0.015*"bundle" + 0.014*"lenses" + 0.013*"came" + 0.013*"package"
words' topics: [(66, [2, 3]), (187, [2, 1]), (393, [2, 3]), (408, [2, 3]), (665, [2, 3]), (2435, [3, 2])]
-------------- 

review: 3320 

review topics: [(0, 0.542716010748531), (1, 0.3505386163601542), (2, 0.053663362159340276), (3, 0.05308201073197444)]
review topics(highest probs): Topic  0
words related to the topics:
 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0

words related to the topics:
 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0.049*"use" + 0.045*"takes" + 0.044*"pictures" + 0.027*"quality" + 0.019*"good" + 0.017*"beginner"
words' topics: [(66, [0, 3]), (86, [0]), (301, [3])]
-------------- 

review: 3350 

review topics: [(0, 0.21359306042860346), (1, 0.7011701379194242), (2, 0.042510017957173324), (3, 0.04272678369479916)]
review topics(highest probs): Topic  1
words related to the topics:
 0.060*"camera" + 0.022*"would" + 0.018*"like" + 0.016*"recommend" + 0.016*"good" + 0.015*"pictures" + 0.014*"canon" + 0.013*"using" + 0.012*"used" + 0.011*"use"
words' topics: [(0, [1, 0]), (30, [1]), (334, [1, 0]), (509, [0, 1]), (1367, [1])]
-------------- 

review: 3351 

review topics: [(0, 0.04411566343217856), (1, 0.0430703137981705), (2, 0.04272492486155046), (3, 0.8700890979081004)]
review topics(highest probs): Topic  3
words related to the topics:
 0.056*"bought" + 0.052*"camera" + 0.041*"loves" + 0.039*"best" + 0.033*

words related to the topics:
 0.060*"camera" + 0.022*"would" + 0.018*"like" + 0.016*"recommend" + 0.016*"good" + 0.015*"pictures" + 0.014*"canon" + 0.013*"using" + 0.012*"used" + 0.011*"use"
words' topics: [(187, [1]), (2027, [0, 3])]
-------------- 

review: 3375 

review topics: [(0, 0.03930427760484834), (1, 0.036752171547834736), (2, 0.7268662204283421), (3, 0.19707733041897488)]
review topics(highest probs): Topic  2
words related to the topics:
 0.070*"camera" + 0.068*"great" + 0.026*"lens" + 0.025*"price" + 0.016*"get" + 0.015*"good" + 0.015*"bundle" + 0.014*"lenses" + 0.013*"came" + 0.013*"package"
words' topics: [(0, [2, 3]), (28, [2]), (72, [2]), (301, [3]), (405, [2, 0]), (591, [2])]
-------------- 

review: 3376 

review topics: [(0, 0.046614475678651235), (1, 0.4603108927352404), (2, 0.4505908332980291), (3, 0.04248379828807939)]
review topics(highest probs): Topic  1
words related to the topics:
 0.060*"camera" + 0.022*"would" + 0.018*"like" + 0.016*"recommend" + 0.016*"g

words related to the topics:
 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0.049*"use" + 0.045*"takes" + 0.044*"pictures" + 0.027*"quality" + 0.019*"good" + 0.017*"beginner"
words' topics: [(0, [0]), (134, [0]), (268, [0]), (405, [0]), (429, [0])]
-------------- 

review: 3407 

review topics: [(0, 0.6380626621873505), (1, 0.04273417177998637), (2, 0.2764305589826827), (3, 0.04277260704998039)]
review topics(highest probs): Topic  0
words related to the topics:
 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0.049*"use" + 0.045*"takes" + 0.044*"pictures" + 0.027*"quality" + 0.019*"good" + 0.017*"beginner"
words' topics: [(66, [0, 2]), (93, [0, 2]), (143, [0]), (377, [2]), (410, [0])]
-------------- 

review: 3408 

review topics: [(0, 0.6513936274655432), (1, 0.052215002252138175), (2, 0.24295346776079238), (3, 0.05343790252152629)]
review topics(highest probs): Topic  0
words related to the topics:
 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.05

review: 3434 

review topics: [(0, 0.05828525062873786), (1, 0.0510238956698188), (2, 0.05469340380081544), (3, 0.8359974499006279)]
review topics(highest probs): Topic  3
words related to the topics:
 0.056*"bought" + 0.052*"camera" + 0.041*"loves" + 0.039*"best" + 0.033*"daughter" + 0.029*"got" + 0.027*"gift" + 0.024*"great" + 0.021*"photography" + 0.021*"buy"
words' topics: [(0, [3, 0]), (93, [3, 0, 2]), (701, [3]), (1008, [3])]
-------------- 

review: 3435 

review topics: [(0, 0.2789917172165977), (1, 0.04265116289234408), (2, 0.04246582855311315), (3, 0.6358912913379451)]
review topics(highest probs): Topic  3
words related to the topics:
 0.056*"bought" + 0.052*"camera" + 0.041*"loves" + 0.039*"best" + 0.033*"daughter" + 0.029*"got" + 0.027*"gift" + 0.024*"great" + 0.021*"photography" + 0.021*"buy"
words' topics: [(0, [3, 0]), (9, [3, 0]), (257, [0, 3]), (701, [3]), (1008, [3])]
-------------- 

review: 3436 

review topics: [(0, 0.8445266793769797), (1, 0.051063996356424404), 

In [43]:
print("Unverified Topics:")
for idx_unverified, topic_unverified in lda_model_unverified2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_unverified, topic_unverified))
print("\n")
print("Verified Topics:")
for idx_verified, topic_verified in lda_model_verified4.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Unverified Topics:
Topic: 0 
Words: 0.035*"camera" + 0.021*"quality" + 0.017*"take" + 0.017*"use" + 0.015*"pictures" + 0.012*"would" + 0.011*"great" + 0.010*"takes" + 0.010*"get" + 0.010*"good"
Topic: 1 
Words: 0.032*"camera" + 0.020*"great" + 0.019*"lens" + 0.013*"easy" + 0.012*"better" + 0.010*"first" + 0.010*"canon" + 0.010*"love" + 0.010*"kit" + 0.010*"need"


Verified Topics:
Topic: 0 
Words: 0.103*"great" + 0.087*"easy" + 0.075*"camera" + 0.050*"love" + 0.049*"use" + 0.045*"takes" + 0.044*"pictures" + 0.027*"quality" + 0.019*"good" + 0.017*"beginner"
Topic: 1 
Words: 0.060*"camera" + 0.022*"would" + 0.018*"like" + 0.016*"recommend" + 0.016*"good" + 0.015*"pictures" + 0.014*"canon" + 0.013*"using" + 0.012*"used" + 0.011*"use"
Topic: 2 
Words: 0.070*"camera" + 0.068*"great" + 0.026*"lens" + 0.025*"price" + 0.016*"get" + 0.015*"good" + 0.015*"bundle" + 0.014*"lenses" + 0.013*"came" + 0.013*"package"
Topic: 3 
Words: 0.056*"bought" + 0.052*"camera" + 0.041*"loves" + 0.039*"best" + 0.

__As we can see the contents between unverified and verified topics, both unverified and verified customers' put high values on this product. And a slightly different between these two is that unverified customers talk about the functions of the product as the words, "take", "lens" and "picture" have relatively higher probability than other words do; on the other hand, verified customers tend to praise the product since the words, "love", "great" and "easy" have relatively higher probability than other words do.__

In [64]:
lda_list_verified_member=[]
lda_list_verified_not_member=[]
lda_list_unverified_member=[]
lda_list_unverified_not_member=[]
for i in js_lda.keys():
    if js_lda[i]["author_attributes"] == []:
        lda_list_unverified_not_member.append(js_lda[i]["review"])
    elif "Verified" in js_lda[i]["author_attributes"][0]:
        if len(js_lda[i]["author_attributes"])>1:
            lda_list_verified_member.append(js_lda[i]["review"]) 
        else:
            lda_list_verified_not_member.append(js_lda[i]["review"])
    else:
        lda_list_unverified_member.append(js_lda[i]["review"])

In [65]:
len(lda_list_verified_member),len(lda_list_verified_not_member),len(lda_list_unverified_member),len(lda_list_unverified_not_member)

(3401, 58, 19, 12)

In [66]:
dict_verified_not_member = corpora.Dictionary(lda_list_verified_not_member)
print(dict_verified_not_member)
doc_term_matrix_verified_not_member = [dict_verified_not_member.doc2bow(doc) for doc in lda_list_verified_not_member]
# print(doc_term_matrix_verified)

Dictionary(277 unique tokens: ['camera', 'great', 'people', 'want', 'start']...)


In [67]:
lda_model_verified_not_member2 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified_not_member, num_topics=2, id2word = dict_verified_not_member, passes=50)

In [68]:
for idx_verified, topic_verified in lda_model_verified_not_member2.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.045*"camera" + 0.035*"love" + 0.028*"great" + 0.020*"quality" + 0.015*"photography" + 0.012*"get" + 0.010*"used" + 0.010*"also" + 0.009*"well" + 0.009*"got"
Topic: 1 
Words: 0.053*"camera" + 0.047*"great" + 0.033*"easy" + 0.022*"good" + 0.019*"use" + 0.016*"love" + 0.015*"pictures" + 0.015*"takes" + 0.013*"using" + 0.010*"would"


In [69]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified_not2 = pyLDAvis.gensim.prepare(lda_model_verified_not_member2, doc_term_matrix_verified_not_member ,dict_verified_not_member)
vis_verified_not2

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.066116  0.0       1        1  54.983555
0     -0.066116  0.0       2        1  45.016445, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
22    Default   5.000000      quality   5.000000  30.0000  30.0000
5     Default  11.000000         easy  11.000000  29.0000  29.0000
100   Default   3.000000          get   3.000000  28.0000  28.0000
190   Default   4.000000        takes   4.000000  27.0000  27.0000
17    Default  13.000000         love  13.000000  26.0000  26.0000
57    Default   6.000000          use   6.000000  25.0000  25.0000
14    Default   2.000000         well   2.000000  24.0000  24.0000
77    Default   2.000000          got   2.000000  23.0000  23.0000
15    Default   2.000000   absolutely   2.000000  22.0000  22.0000
175   Default   3.000000        would   3.000000  21.0000  21.0000
177   Default   3.000000       really   3.000000  20.0000  20.0000
10    Default   5.000000  photography   5.000000  19.0000  19.0000
4     Default   1.000000        start   1.000000  18.0000  18.0000
60    Default   1.000000         film   1.000000  17.0000  17.0000
76    Default   1.000000       photos   1.000000  16.0000  16.0000
8     Default   1.000000       simple   1.000000  15.0000  15.0000
68    Default   1.000000       allows   1.000000  14.0000  14.0000
244   Default   1.000000        focus   1.000000  13.0000  13.0000
146   Default   1.000000          far   1.000000  12.0000  12.0000
245   Default   1.000000     starting   1.000000  11.0000  11.0000
230   Default   1.000000        close   1.000000  10.0000  10.0000
61    Default   3.000000         used   3.000000   9.0000   9.0000
106   Default   5.000000     pictures   5.000000   8.0000   8.0000
37    Default   3.000000         also   3.000000   7.0000   7.0000
191   Default   2.000000         long   2.000000   6.0000   6.0000
165   Default   2.000000         came   2.000000   5.0000   5.0000
38    Default   2.000000         like   2.000000   4.0000   4.0000
180   Default   2.000000    recommend   2.000000   3.0000   3.0000
219   Default   2.000000        happy   2.000000   2.0000   2.0000
143   Default   2.000000        learn   2.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
30     Topic2   0.970359         rely   1.303638   0.5029  -5.5067
25     Topic2   0.970359       helper   1.303638   0.5029  -5.5067
26     Topic2   0.970359         show   1.303638   0.5029  -5.5067
24     Topic2   0.970359          big   1.303638   0.5029  -5.5067
9      Topic2   0.970359          pet   1.303638   0.5029  -5.5067
16     Topic2   0.970359         fell   1.303638   0.5029  -5.5067
21     Topic2   0.970359    regarding   1.303638   0.5029  -5.5067
23     Topic2   0.970359      editing   1.303638   0.5029  -5.5067
28     Topic2   0.970359        style   1.303638   0.5029  -5.5067
2      Topic2   0.970359       people   1.303638   0.5029  -5.5067
20     Topic2   0.970359  compliments   1.303638   0.5029  -5.5067
11     Topic2   0.970359       nature   1.303638   0.5029  -5.5067
29     Topic2   0.970359       always   1.303638   0.5029  -5.5067
6      Topic2   0.970359   understand   1.303638   0.5029  -5.5067
61     Topic2   2.354397         used   3.261118   0.4724  -4.6203
17     Topic2   8.387416         love  13.073631   0.3543  -3.3499
10     Topic2   3.559992  photography   5.223649   0.4147  -4.2069
37     Topic2   2.281406         also   3.263184   0.4402  -4.6518
0      Topic2  10.709675       camera  26.318940  -0.1010  -3.1055
91     Topic2   1.659525        phone   2.615235   0.3433  -4.9701
126    Topic2   1.641375      perfect   2.615748   0.3321  -4.9811
50     Topic2   1.627469        canon   2.616142   0.3235  -4.9896
33     Topic2   1.627379          new   2.616144   0.3234  -4.9896
18     To

In [70]:
dict_verified_member = corpora.Dictionary(lda_list_verified_member)
print(dict_verified_member)
doc_term_matrix_verified_member = [dict_verified_member.doc2bow(doc) for doc in lda_list_verified_member]
# print(doc_term_matrix_verified)

Dictionary(2418 unique tokens: ['camera', 'full', 'featured', 'entry', 'level']...)


In [71]:
lda_model_verified_member4 = gensim.models.ldamodel.LdaModel(doc_term_matrix_verified_member, num_topics=4, id2word = dict_verified_member, passes=50)

In [72]:
for idx_verified, topic_verified in lda_model_verified_member4.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx_verified, topic_verified))

Topic: 0 
Words: 0.037*"camera" + 0.031*"best" + 0.029*"canon" + 0.025*"happy" + 0.024*"picture" + 0.020*"one" + 0.018*"love" + 0.016*"rebel" + 0.015*"purchased" + 0.014*"quality"
Topic: 1 
Words: 0.076*"camera" + 0.030*"bought" + 0.023*"great" + 0.022*"would" + 0.022*"recommend" + 0.021*"loves" + 0.019*"first" + 0.019*"really" + 0.017*"daughter" + 0.014*"photography"
Topic: 2 
Words: 0.127*"easy" + 0.113*"great" + 0.074*"use" + 0.063*"takes" + 0.061*"camera" + 0.045*"love" + 0.044*"pictures" + 0.029*"quality" + 0.022*"good" + 0.015*"amazing"
Topic: 3 
Words: 0.072*"camera" + 0.060*"great" + 0.025*"good" + 0.022*"price" + 0.020*"pictures" + 0.016*"lens" + 0.014*"nice" + 0.014*"get" + 0.013*"take" + 0.011*"lenses"


In [74]:
# A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.
# A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.
pyLDAvis.enable_notebook()
vis_verified_member4 = pyLDAvis.gensim.prepare(lda_model_verified_member4, doc_term_matrix_verified_member ,dict_verified_member)
vis_verified_member4

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.004875 -0.021201       1        1  33.321597
1      0.042720 -0.181190       2        1  30.637334
2     -0.239922  0.075253       3        1  20.095042
0      0.202078  0.127138       4        1  15.946027, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
86    Default   846.000000         easy   846.000000  30.0000  30.0000
93    Default  1651.000000        great  1651.000000  29.0000  29.0000
410   Default   420.000000        takes   420.000000  28.0000  28.0000
134   Default   571.000000          use   571.000000  27.0000  27.0000
223   Default   270.000000         best   270.000000  26.0000  26.0000
143   Default   590.000000     pictures   590.000000  25.0000  25.0000
438   Default   247.000000        price   247.000000  24.0000  24.0000
405   Default   561.000000         love   561.000000  23.0000  23.0000
511   Default   222.000000    recommend   222.000000  22.0000  22.0000
135   Default   233.000000        canon   233.000000  21.0000  21.0000
983   Default   215.000000        loves   215.000000  20.0000  20.0000
409   Default   135.000000        happy   135.000000  19.0000  19.0000
95    Default   130.000000      picture   130.000000  18.0000  18.0000
301   Default   371.000000       bought   371.000000  17.0000  17.0000
121   Default   362.000000      quality   362.000000  16.0000  16.0000
404   Default   250.000000        would   250.000000  15.0000  15.0000
701   Default   171.000000     daughter   171.000000  14.0000  14.0000
18    Default   214.000000        first   214.000000  13.0000  13.0000
665   Default   156.000000         nice   156.000000  12.0000  12.0000
231   Default    90.000000        rebel    90.000000  11.0000  11.0000
8     Default   217.000000          one   217.000000  10.0000  10.0000
116   Default   256.000000       really   256.000000   9.0000   9.0000
230   Default   141.000000    purchased   141.000000   8.0000   8.0000
295   Default   158.000000  photography   158.000000   7.0000   7.0000
66    Default   528.000000         good   528.000000   6.0000   6.0000
72    Default   196.000000         lens   196.000000   5.0000   5.0000
408   Default   114.000000         deal   114.000000   4.0000   4.0000
429   Default   148.000000        still   148.000000   3.0000   3.0000
519   Default   127.000000        learn   127.000000   2.0000   2.0000
509   Default   151.000000      amazing   151.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
977    Topic4     7.837806         open     8.610307   1.7420  -6.5145
1048   Topic4     7.555158      lighter     8.329870   1.7383  -6.5512
219    Topic4     7.541809      learned     8.328408   1.7368  -6.5530
150    Topic4     7.530835      believe     8.329488   1.7352  -6.5544
459    Topic4     7.274919         hope     8.048691   1.7349  -6.5890
171    Topic4    20.107541          day    22.911849   1.7054  -5.5723
243    Topic4    13.945269        given    16.176612   1.6875  -5.9383
302    Topic4    15.997988      replace    18.930885   1.6676  -5.8010
135    Topic4   154.564561        canon   233.261042   1.4244  -3.5328
223    Topic4   165.960993         best   270.359222   1.3480  -3.4617
522    Topic4    37.017345       simple    51.934062   1.4974  -4.9621
230    Topic4    80.688398    purchased   141.600051   1.2735  -4.1828
323    Topic4    20.501782         mode    26.517903   1.5787  -5.5529
8      Topic4   104.589257          one   217.354706   1.1045  -3.9234
164    Topic4    67.309205         used   130.533646   1.1736  -4.3641
169    Topic4    18.863710         back    25.941073   1.5174  -5.6362
120    Topic4    42.508804     purchase    85.505955   1.1371  -4.8237
157    Topic4    20.746469      feature    30.395351   1.4540  -5.5411
315    Topic